# Using `siibra-python` to learn gene regulatory networks

`siibra-python` allows users to query gene expression data from the Allen brain atlas. [Their documentation](https://siibra-python.readthedocs.io/en/latest/examples/03_data_features/004_gene_expressions.html) provides a full explanation on how it works. 

First of all, you should determine the atlas, region and genes of interest. After that, you can use `siibra`'s `get_features` to retrieve the desired data.

In [ ]:
import siibra
import pandas as pd
from neurogenpy import BayesianNetwork

atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS
region = atlas.get_region("V1")

genes = ["A_23_P64051", "ATP5G1", "RAB33B"]


samples = {gene_name: [statistics.mean(f.expression_levels) for 
                       f in siibra.get_features(region, 'gene', 
                                                gene=gene_name)] for 
           gene_name in genes}

Finally, you can learn the network with the `BayesianNetwork` class after creating a `pandas DataFrame`.

In [ ]:
df = pd.DataFrame(samples)

bn = BayesianNetwork().fit(df, algorithm="fges_merge", estimation="mle")